In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [6]:
# Read in json file with new lines format
df = pd.read_json('../../datasets/recipes_with_servings_and_normalized_nutrients.json', lines=True)
df.head()

,fsa_lights_per100g,id,ingredients,instructions,nutr_per_ingredient,nutr_values_per100g,partition,quantity,title,unit,...,status,found,error,n_servings,total_fat_per_serving,total_pro_per_serving,total_sod_per_serving,total_sat_per_serving,total_sug_per_serving,total_nrg_per_serving
0,"{'fat': 'orange', 'salt': 'green', 'saturates'...",0006ca31f4,"[{'text': 'cocoa, dry powder, unsweetened'}, {...",[{'text': 'Measure out the cocoa powder into a...,"[{'fat': 141.36, 'nrg': 2352.0, 'pro': 202.32,...","{'energy': 303.4353995519, 'fat': 5.0948469007...",train,"[{'text': '12'}, {'text': '12'}, {'text': '1'}]",Natural Peanut Butter Chocolate Bon Bons,"[{'text': 'cup'}, {'text': 'cup'}, {'text': 'c...",...,200,True,NaN,12.0,22.740000,22.620000,35.333333,9.142667,282.130000,1354.333333
1,"{'fat': 'green', 'salt': 'red', 'saturates': '...",00073a6b36,"[{'text': 'cheese, pasteurized process, americ...","[{'text': 'Put milk, onion salt and pepper sau...","[{'fat': 54.0738363, 'nrg': 631.05987, 'pro': ...","{'energy': 57.1235315011, 'fat': 2.47764100560...",train,"[{'text': '6'}, {'text': '13'}, {'text': '14'}...",Easy Cheese Sauce,"[{'text': 'ounce'}, {'text': 'cup'}, {'text': ...",...,200,True,NaN,4.0,21.221703,34.425175,9195.832618,12.699995,42.204004,489.279367
2,"{'fat': 'orange', 'salt': 'red', 'saturates': ...",0007f5f800,"[{'text': 'cornmeal, degermed, unenriched, yel...","[{'text': 'In a large skillet, stir the cornme...","[{'fat': 2.75, 'nrg': 581.0, 'pro': 11.16, 'sa...","{'energy': 399.5182190719, 'fat': 18.337440469...",train,"[{'text': '1'}, {'text': '3/4'}, {'text': '3/4...",Cornmeal Crackers with Pumpkin Seeds,"[{'text': 'cup'}, {'text': 'cup'}, {'text': 'c...",...,200,True,NaN,2.0,41.392187,16.085625,2352.156250,22.800094,6.873437,901.812500
3,"{'fat': 'green', 'salt': 'green', 'saturates':...",0008c6e39a,"[{'text': 'sugars, powdered'}, {'text': 'wheat...",[{'text': 'Move oven rack to lowest position.'...,"[{'fat': 0.0, 'nrg': 700.5, 'pro': 0.0, 'sat':...","{'energy': 149.4259987004, 'fat': 0.3679178142...",train,"[{'text': '1 1/2'}, {'text': '3/4'}, {'text': ...",Chocolate-Orange Angel Food Cake,"[{'text': 'cup'}, {'text': 'cup'}, {'text': 'c...",...,200,True,NaN,1.0,5.945000,57.592500,1201.500000,2.045500,442.950000,2414.500000
4,"{'fat': 'orange', 'salt': 'red', 'saturates': ...",0009785a2e,"[{'text': 'rice, white, long-grain, regular, u...",[{'text': 'Add rice to a large pan of boiling ...,"[{'fat': 0.44, 'nrg': 205.0, 'pro': 4.25, 'sat...","{'energy': 57.3496659243, 'fat': 3.4322726627,...",test,"[{'text': '1'}, {'text': '3'}, {'text': '3'}, ...","Rice With Lemon, Capers & Parsley","[{'text': 'cup'}, {'text': 'cup'}, {'text': 't...",...,200,True,NaN,4.0,9.053906,2.161406,7085.093750,5.533250,0.194531,151.281250


In [11]:
# Build a text parser to remove drinks, sauces, sides, appetizers, and breads. Do this simply by parsing the title for keywords.
text_parser = lambda x: any([kw in x.lower() for kw in [
    'drink', 'sauce', 'side', 'appetizer', 'bread', 'cocktail', 'smoothie', 'dip', 
    'pudding', 'vinaigrette', 'salad dressing', 'salad dressing', 'margarita', 
    'lemonade', 'iced tea', 'juice', 'soda', 'beer', 'wine', 'cocktail', 'mocktail', 
    'tea', 'coffee', 'espresso', 'latte', 'cappuccino', 'mocha', 'frappuccino',
    'hot chocolate', 'chocolate milk', 'milkshake', 'dressing', 'gravy', 'broth',
    'float', 'sangria', 'punch', 'slushie', 'slushy', 'fondue', 'baguette', 'bread',
    'relish', 'chutney', 'salsa', 'hummus', 'guacamole', 'bruschetta', 'crostini',
    'crouton', 'focaccia', 'naan', 'pita'
]])

# Filter out these recipes
recipe_subset = df[~df['title'].apply(text_parser)].copy()

# Print initial number of recipes, final number of recipes, and number removed
print(f"Initial number of recipes: {len(df)}")
print(f"Final number of recipes: {len(recipe_subset)}")
print(f"Number of recipes removed: {len(df) - len(recipe_subset)}")

Initial number of recipes: 37225
Final number of recipes: 27213
Number of recipes removed: 10012


In [12]:
# Save to new json file
recipe_subset.to_json('../../datasets/recipes_with_servings_and_normalize_nutrients_filtered.json', orient='records', lines=True)